# Data Cleaning - How to Remove Outliers

## 0. Introduction

This contebook contains:
  1. Standard Deviation Method
  2. Interquartile Range Method
  3. Automatic Outlier Detection

## 1. Standard Deviation Method

In [4]:
import numpy as np

data = 5 * np.random.randn(10000) + 50
data_mean, data_std = np.mean(data), np.std(data)
cutoff = data_std*3
upper, lower = data_mean + cutoff, data_mean-cutoff

outliers = [x for x in data if x<lower or x>upper]
print(f"Identified outliers: {len(outliers)}")

Identified outliers: 33


In [6]:
removed_outliers = [x for x in data if x<upper and x>lower]
print(f"Non-Outliers observations: {len(removed_outliers)}")

Non-Outliers observations: 9967


## 2. Interquartile Range Method

In [7]:
data = 5 * np.random.randn(10000) + 50
q25, q75 = np.percentile(data, 25), np.percentile(data, 75)
iqr = q75 - q25
print(f"25th percentile: {q25}\n75th percentile: {q75}\nIQR :{iqr}")

25th percentile: 46.63350703576179
75th percentile: 53.33959433595811
IQR :6.706087300196323


In [8]:
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in data if x<lower or x>upper]
print(f"Identified Outliers: {len(outliers)}")

Identified Outliers: 77


In [9]:
removed_outliers = [x for x in data if x<upper and x>lower]
print(f"Non-Outlier observations: {len(removed_outliers)}")

Non-Outlier observations: 9923


## 3. Automatic Outlier Detection

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error

In [12]:
path = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv"
data = pd.read_csv(path, header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [13]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

In [15]:
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
mask = yhat != -1
X_train, y_train = X_train.iloc[mask,:], y_train.iloc[mask]
X_train.shape, y_train.shape

((371, 13), (371,))

In [16]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

yhat = linear_reg.predict(X_test)

mae = mean_absolute_error(y_test, yhat)
print(f"MAE: {mae:.2f}")

MAE: 3.21
